In [1]:
import numpy as np
import pyqtgraph as pqg
%gui qt
pqg.setConfigOption('antialias', True)

In [2]:
def axs_format(ax):
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')
    
def get_crossings(v, thr=3.8):
    from numpy import where
    return np.where((v[:-1] < thr) * (v[1:] > thr))[0] 


def load_matfile(fname, dset='rawdata'):
    from h5py import File
    result = dict()
    with File(fname, 'r') as f:
        for x in f[dset].keys():
            result[x] = np.squeeze(f[dset][x].value)
            
    return result

In [ ]:
#import sys
#lib = 'C:\\Users\\bennettd\\Documents\\GitHub\\fish\\'
#if lib not in sys.path:
#    sys.path.append(lib)

In [34]:
import fish.ephys.ephys as ep
from glob import glob
from skimage.io import imread

base_dirs = ['/groups/ahrens/ahrenslab/FROM_TIER2/Yu/SPIM/active_datasets/20170221/fish4/20170221_4_4_gfaprgeco_hucgc_6dpf_wholebrain_faster_brighter_20170221_185519/',
            '/groups/ahrens/ahrenslab/FROM_TIER2/Yu/SPIM/active_datasets/20170131/fish5/20170131_5_1_GFAPRGECO_DBHGC5slow_5dpf_waist_CLPause_0GAIN_GAINMODULATION_20170131_220542/',
            '/groups/ahrens/ahrenslab/davis/data/spim/proc/20160607/5dpf_cy171xcy221_f1_flash_1_20160607_205245/proj/',
            '/groups/ahrens/ahrenslab/FROM_TIER2/Yu/SPIM/active_datasets/20170228/fish4/20170228_4_1_gfaprgeco_hucgc_6dpf_shorttrials_20170228_185002/']

exp = -1
dirs = {}
dirs['base'] = base_dirs[exp]
dirs['proj'] = dirs['base'] + 'projections/'
dirs['ep'] = dirs['base'] + 'ephys/'

fnames = glob(dirs['proj'] + 't_*.tif')
fnames.sort()

ep_dat = load_matfile(dirs['ep'] + 'rawdata.mat')
power = np.array([ep.windowed_variance(ep_dat['ch1'])[0], ep.windowed_variance(ep_dat['ch2'])[0]])
cam = get_crossings(ep_dat['ch3'])
print(len(fnames))
cam = cam[:len(fnames)]

7200


In [5]:
%%time
import numpy as np
plr = slice(0, 3000)
imdat = np.array([imread(f) for f in fnames])
print(imdat.shape)

(7200, 8, 512, 512)
CPU times: user 3min 59s, sys: 26.5 s, total: 4min 25s
Wall time: 10min 56s


In [6]:
imdat = imdat.reshape(imdat.shape[0], imdat.shape[1] * imdat.shape[2], imdat.shape[3])

In [7]:
pqim = pqg.image(imdat)

In [8]:
ts = dict()

In [35]:
ts['ephys'] = power[0][cam] + power[1][cam]

In [40]:
ts['6pack'] = pqim.roi.getArrayRegion(imdat, pqim.imageItem, axes=(1,2)).mean((1,2))

In [13]:
print(ts.keys())

dict_keys(['lc', 'waist_n', 'mo', 'waist_g'])


In [42]:
from scipy.stats import zscore
pl = pqg.plot(title='ROI timeseries')
leg = pl.plotItem.addLegend()
to_plot = ['6pack','waist_g']

n_lines = len(to_plot) + 2

fs_ep = 6000
ds_ep = 20
fs_im = np.median(np.diff(cam)) * fs_ep

plr_im = np.arange(0, imdat.shape[0])
plr_ep = np.arange(cam[plr_im[0]], cam[plr_im[-1]], ds_ep)
x_axis_im = cam[plr_im] / fs_ep 
x_axis_ep = plr_ep / fs_ep

pl.plot(x_axis_ep, .1 * zscore(power[0] + power[1])[plr_ep], pen=(len(to_plot), n_lines), autodownsample=True, name='swim power')
pl.plot(x_axis_ep,  zscore(ep_dat['stimParam3'])[plr_ep], pen=(len(to_plot) + 1, n_lines), autodownsample=True, name='stim condition')
for ind,k in enumerate(to_plot):
    pl.plot(x_axis_im, zscore(ts[k])[plr_im], pen=(ind, n_lines), name=k, autodownsample=True)

In [ ]:
import matplotlib
matplotlib.rcParams.update({'font.size' : 14})

plr = np.arange(1600, 1840)

condition = ep_dat['stimParam3'][cam]

fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(16,8))
axs.plot(plr * fs_im, ts['waist_g'][plr], label='glial waist', color='r', linewidth=2)
axs.plot(plr * fs_im, ts['lc'][plr], label='locus ceroeleus', color='b', linewidth=2, alpha = .7)
axs.plot(plr * fs_im, 1000 + 10 * condition[plr], label = 'stimulus condition', zorder=0, alpha = .8, color='k')
axs.plot(plr * fs_im, 1000 + 300000 * ts['ephys'][plr], label='ventral root', color='m', linewidth=2, alpha=.7)
axs.legend()
axs.set_xlim(plr[0]*fs_im, plr[-1] * fs_im)
axs.set_xlabel('Time [s]')
axs_format(axs)
plt.savefig('X:\\davis\\tmp\\putative_gu_lc.png')

In [ ]:
flat = np.squeeze(imdat.reshape(imdat.shape[0], imdat.shape[1]*imdat.shape[2]))

In [ ]:
flat = flat[:, np.where(flat[::100,:].std(0) != 0)[0]]

In [ ]:
flat.shape

## Save ROI

In [ ]:
from sklearn.decomposition import pca

In [ ]:
roi_data = pqim.roi.getArrayRegion(imdat, pqim.imageItem, axes=(1,2))

In [ ]:
flat = roi_data.reshape(roi_data.shape[0], roi_data.shape[1] * roi_data.shape[2])

In [ ]:
pca_result = pca.PCA(n_components=5).fit(flat.T)

In [ ]:
pca_result.components_.shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(pca_result.components_[0])